### Cleaning and Processing Data

In [1]:
# Importing required libraries
import pandas as pd
import os
import datetime
import math
import numpy as np
from sqlalchemy import create_engine
from config import username, password
import requests
import json

In [2]:
# Reading CSV files
data = 'data/data.csv'
genres = 'data/data_w_genres.csv'
mus_data = pd.read_csv(data)
genres_data = pd.read_csv(genres)

In [3]:
# Cleaning genres column's str values
genres_data['genres'] = genres_data['genres'].str.replace(r"[\"\])([']", '')
genres_data['artists'] = genres_data['artists'].str.replace(r"[\"\])(*[']", '')
genres_data['artists'] = genres_data['artists'].str.replace(r"[$]", 'S')

In [4]:
# Keeping only selected columns for genres
genres_data = genres_data[['artists', 'genres']]

In [5]:
# Cleaning main data column's str values
mus_data['artists'] = mus_data['artists'].str.replace(r"[\"\])(*[']", '')
mus_data['artists'] = mus_data['artists'].str.replace(r"$", 'S')

In [6]:
# Creating decades column to sort data by 10 years intervals
mus_data['decade'] = np.floor(mus_data['year']/10)*10

In [7]:
# Assigning Int64 to decades column values
mus_data = mus_data.astype({"decade": int})

In [8]:
# Adding additional artist_songs column to get accurate song's counts per decades
mus_data['artist_song'] = mus_data['artists'] + ' - ' + mus_data['name']

In [9]:
# Merging dataframes - genres data with main data
all_data = mus_data.merge(genres_data, on='artists', how='left')
# merged_data.shape

In [10]:
# Replacing NaN values with empty values after merging
all_data['genres'].fillna('---', inplace=True)
# merged_data.shape

In [11]:
# Sorting data by duration in order to drop non-music items
all_data = all_data.sort_values('duration_ms',ascending=False)

In [12]:
# Cleaning merged data, dropping audio books
all_data.drop(all_data.loc[all_data['duration_ms'] >= 1000000].index, inplace=True)
all_data.drop(all_data.loc[all_data['artists'] == 'Georgette Heyer, Irina Salkow'].index, inplace=True)
all_data.drop(all_data.loc[all_data['artists'] == 'Трумен Капоте'].index, inplace=True)
all_data.drop(all_data.loc[all_data['artists'] == 'Seweryn Goszczyński'].index, inplace=True)
all_data.drop(all_data.loc[all_data['artists'] == 'H.P. Lovecraft'].index, inplace=True)
all_data.drop(all_data.loc[all_data['artists'] == 'Эрих Мария Ремарк'].index, inplace=True)
all_data.drop(all_data.loc[all_data['artists'] == 'Эрнест Хемингуэй'].index, inplace=True)

In [13]:
all_data = all_data.drop(['release_date'], axis = 1)

In [14]:
all_data.shape

(167017, 21)

In [15]:
all_data = all_data.drop_duplicates(subset='artist_song', keep="first")

In [16]:
all_data.shape

(154081, 21)

In [17]:
# Rearranging and renaming columns order
clean_data_all = all_data[['decade', 'year', 'artists', 'name', 'genres', 'valence', 'acousticness', 'danceability', 'energy', 'explicit', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'popularity', 'speechiness', 'tempo', 'id']]

In [18]:
clean_data_all.columns = ['Decade', 'Year', 'Artist', 'Song', 'Genre', 'Valence', 'Acousticness', 'Danceability', 'Energy', 'Explicit', 'Instrumentalness', 'Key', 'Liveness', 'Loudness', 'Mode', 'Popularity', 'Speechiness', 'Tempo', 'IDs']

In [19]:
clean_data_all.reset_index(drop=True, inplace=True)

In [20]:
# clean_data_all = clean_data_all.replace(',','', regex=True)

In [21]:
# clean_data_all.astype({'Song': 'string'}).dtypes

In [22]:
clean_data_all = clean_data_all.replace({'/':''}, regex=True)

In [23]:
clean_data_all.head(2)

,Decade,Year,Artist,Song,Genre,Valence,Acousticness,Danceability,Energy,Explicit,Instrumentalness,Key,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,IDs
0,1940,1940,"Various Artists, Igor Stravinsky",A propos of Le Sacre - Commentary by Igor Stra...,---,0.405,0.35,0.635,0.110,0,0.0000,1,0.383,-30.467,0,0,0.930,85.650,431ryeJJoLFxBX1iSJFMde
1,1970,1977,Fela Kuti,Opposite People,"afrobeat, afropop, funk, soul, world",0.904,0.11,0.668,0.897,0,0.0543,5,0.123,-5.167,0,25,0.177,117.973,2nsQdjrZesTsFqVpYNBpUm


In [24]:
# Saving master clean data to csv file
clean_data_all.to_csv('data/clean_data_all.csv', encoding='utf-8', index=False)

In [25]:
# Creating Posgres Engine

pg_user = username
pg_password = password

In [27]:
connection_string = f"{pg_user}:{pg_password}@localhost:5432/spotifyDB"
engine = create_engine(f'postgresql://{connection_string}')
engine.table_names()

['songs']

In [28]:
clean_data_all.to_sql(name='songs', con=engine, if_exists='replace', index=False)

In [29]:
pd.read_sql_query('select * from songs', con=engine).head()

,Decade,Year,Artist,Song,Genre,Valence,Acousticness,Danceability,Energy,Explicit,Instrumentalness,Key,Liveness,Loudness,Mode,Popularity,Speechiness,Tempo,IDs
0,1940,1940,"Various Artists, Igor Stravinsky",A propos of Le Sacre - Commentary by Igor Stra...,---,0.405,0.350,0.635,0.110,0,0.0000,1,0.3830,-30.467,0,0,0.9300,85.650,431ryeJJoLFxBX1iSJFMde
1,1970,1977,Fela Kuti,Opposite People,"afrobeat, afropop, funk, soul, world",0.904,0.110,0.668,0.897,0,0.0543,5,0.1230,-5.167,0,25,0.1770,117.973,2nsQdjrZesTsFqVpYNBpUm
2,1950,1956,"D. Brubeck, The Dave Brubeck Quartet",Elementals,---,0.273,0.804,0.255,0.446,0,0.0266,9,0.0947,-11.226,0,4,0.0420,78.110,2ieGcEuaTNlp6XQ1hzNtUA
3,1940,1949,"André Gide, Jean Amrouche, Arthur Rubinstein","""Saül""",---,0.399,0.944,0.565,0.300,0,0.0000,10,0.2140,-23.942,1,0,0.9080,60.708,3F5BU0Uu6udC41I7jp3Wuu
4,1960,1969,Miles Davis,Mademoiselle Mabry - New Mix,"bebop, contemporary post-bop, cool jazz, hard ...",0.155,0.879,0.561,0.157,0,0.7950,5,0.0995,-16.234,0,22,0.0425,111.844,3OvO7qmeDemKrjGo7I542f


### (OBSOLETE) Dividing data into decades to find out top-10 artists

In [ ]:
# 1920s decade

# mus_1920s = merged_data[merged_data['Decade']==1920]
# top10songs20s = mus_1920s.sort_values(['Popularity'], ascending=False)
# top10songs20s.head(5)

In [ ]:
# a20 = mus_1920s['Artist/Band'].value_counts().sort_values(ascending=False).head(10)
# top10artists20s = pd.DataFrame({'Artist/Band':a20.index}).merge(mus_1920s, how='left')
# top10artists20s['Artist/Band'].value_counts().nlargest(10)

In [ ]:
# 1930s decade

# mus_1930s = merged_data[merged_data['Decade']==1930]
# top10songs30s = mus_1930s.sort_values(['Popularity'], ascending=False)
# top10songs30s.head(5)

In [ ]:
# a30 = mus_1930s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists30s = pd.DataFrame({'artists':a30.index}).merge(mus_1930s, how='left')
# top10artists30s['artists'].value_counts().nlargest(10)

In [ ]:
# 1930s decade

# mus_1940s = merged_data[merged_data['decade']==1940]
# top10songs40s = mus_1940s.sort_values(['popularity'], ascending=False)
# top10songs40s.head(5)

In [ ]:
# a40 = mus_1940s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists40s = pd.DataFrame({'artists':a40.index}).merge(mus_1940s, how='left')
# top10artists40s['artists'].value_counts().nlargest(10)

In [ ]:
# 1950s decade

# mus_1950s = merged_data[merged_data['decade']==1950]
# top10songs50s = mus_1950s.sort_values(['popularity'], ascending=False)
# top10songs50s.head(5)

In [ ]:
# a50 = mus_1950s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists50s = pd.DataFrame({'artists':a50.index}).merge(mus_1950s, how='left')
# top10artists50s['artists'].value_counts().nlargest(10)

In [ ]:
# 1960s decade

# mus_1960s = merged_data[merged_data['decade']==1960]
# top10songs60s = mus_1960s.sort_values(['popularity'], ascending=False)
# top10songs60s.head(5)

In [ ]:
# a60 = mus_1960s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists60s = pd.DataFrame({'artists':a60.index}).merge(mus_1960s, how='left')
# top10artists60s['artists'].value_counts().nlargest(10)

In [ ]:
# 1970s decade

# mus_1970s = merged_data[merged_data['decade']==1970]
# top10songs70s = mus_1970s.sort_values(['popularity'], ascending=False)
# top10songs70s.head(5)

In [ ]:
# a70 = mus_1970s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists70s = pd.DataFrame({'artists':a70.index}).merge(mus_1970s, how='left')
# top10artists70s['artists'].value_counts().nlargest(10)

In [ ]:
# 1980s decade

# mus_1980s = merged_data[merged_data['decade']==1980]
# top10songs80s = mus_1980s.sort_values(['popularity'], ascending=False)
# top10songs80s.head(5)

In [ ]:
# a80 = mus_1980s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists80s = pd.DataFrame({'artists':a80.index}).merge(mus_1980s, how='left')
# top10artists80s['artists'].value_counts().nlargest(10)

In [ ]:
# 1990s decade

# mus_1990s = merged_data[merged_data['decade']==1990]
# top10songs90s = mus_1990s.sort_values(['popularity'], ascending=False)
# top10songs90s.head(5)

In [ ]:
# a90 = mus_1990s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists90s = pd.DataFrame({'artists':a90.index}).merge(mus_1990s, how='left')
# top10artists90s['artists'].value_counts().nlargest(10)

In [ ]:
# 2000s decade

# mus_2000s = merged_data[merged_data['decade']==2000]
# top10songs00s = mus_2000s.sort_values(['popularity'], ascending=False)
# top10songs00s.head(5)

In [ ]:
# b00 = mus_2000s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists00s = pd.DataFrame({'artists':b00.index}).merge(mus_2000s, how='left')
# top10artists00s['artists'].value_counts().nlargest(10)

In [ ]:
# 2010s decade

# mus_2010s = merged_data[merged_data['decade']==2010]
# top10songs10s = mus_2010s.sort_values(['popularity'], ascending=False)
# top10songs10s.head(5)

In [ ]:
# b10 = mus_2010s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists10s = pd.DataFrame({'artists':b10.index}).merge(mus_2010s, how='left')
# top10artists10s['artists'].value_counts().nlargest(10)

In [ ]:
# 2020s decade

# mus_2020s = merged_data[merged_data['decade']==2020]
# top10songs20s = mus_2020s.sort_values(['popularity'], ascending=False)
# top10songs20s.head(5)

In [ ]:
# b20 = mus_2020s['artists'].value_counts().sort_values(ascending=False).head(10)
# top10artists20s = pd.DataFrame({'artists':b20.index}).merge(mus_2020s, how='left')
# top10artists20s['artists'].value_counts().nlargest(10)

In [ ]:
## All Decades Data Together

# all_mus = mus_1920s.append([mus_1930s, mus_1940s, mus_1950s, mus_1960s, mus_1970s, mus_1980s, mus_1990s, mus_2000s, mus_2010s, mus_2020s])
# all_mus.to_csv('data/clean_data_decade.csv', encoding='utf-8', index=False)

In [ ]:
# all_mus.shape